In [1]:
!pip install pyPDF2


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from PyPDF2 import PdfReader
import re
import json

# 1. PDF 로드
pdf_path = "../data/금융보안원 - 금융권에 적합한 ISMS-P 인증기준 점검항목 안내서(2023.12.).pdf"
reader = PdfReader(pdf_path)


In [2]:

# 2. 전체 텍스트 추출
full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"

print(full_text)




 P･A･R･T
01정보보호 및 개인정보보호 
관리체계 (ISMS-P) 인증기준 개요· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 1
1. 관리체계 수립 및 운영· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 3
2. 보호대책 요구사항 · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 4
3. 개인정보 처리 단계별 요구사항 · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 6
 P･A･R･T
02정보보호 및 개인정보보호 
관리체계 (ISMS-P) 인증기준 설명· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 9
1. 관리체계 수립 및 운영· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 10
1.1 관리체계 기반 마련· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · 10
1.2 위험 관리· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·

In [3]:
# 조문 단위로 분리 및 정제
def split_articles(text):
    # "제X조(제목)" 형식 찾기
    pattern = r'(제\d+조(?:의\d+)?\([^)]+\))'
    # 분리
    articles = re.split(pattern, text)
    results = []
    for i in range(1, len(articles), 2):
        article_num_title = articles[i]
        article_text = articles[i+1].strip().replace('\n', ' ')
        # '제X조(제목)'에서 번호와 제목 분리
        m = re.match(r'(제\d+조(?:의\d+)?)[(]([^)]+)[)]', article_num_title)
        if m:
            law_num = m.group(1)
            law_title = m.group(2)
            prompt = f"개인정보 보호법 {law_num} \"{law_title}\"에 대해서 설명하라."
            results.append({'instruction': prompt, 'output': article_text})
    return results

In [4]:
split_articles(full_text)

[{'instruction': '개인정보 보호법 제1조 "목적"에 대해서 설명하라.',
  'output': '이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인 의 존엄과 가치를 구현함을 목적으로 한다. <개정 2014. 3. 24.>'},
 {'instruction': '개인정보 보호법 제2조 "정의"에 대해서 설명하라.',
  'output': '이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014. 3. 24., 2020. 2. 4., 2023. 3. 14.> 1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다. 가. 성명, 주민등록번호 및 영상 등을 통하여 개인을 알아볼 수 있는 정보  나. 해당 정보만으로는 특정 개인을 알아볼 수 없더라도 다른 정보와 쉽게 결합하여 알아볼 수 있는 정보. 이 경우  쉽게 결합할 수 있는지 여부는 다른 정보의 입수 가능성 등 개인을 알아보는 데 소요되는 시간, 비용, 기술 등  을 합리적으로 고려하여야 한다.  다. 가목 또는 나목을 제1호의2에 따라 가명처리함으로써 원래의 상태로 복원하기 위한 추가 정보의 사용ㆍ결합  없이는 특정 개인을 알아볼 수 없는 정보(이하 “가명정보”라 한다)  1의2. “가명처리”란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의 방법으로 추가 정보가 없이는 특정 개인을 알아볼 수 없도록 처리하는 것을 말한다.  2. “처리”란 개인정보의 수집, 생성, 연계, 연동, 기록, 저장, 보유, 가공, 편집, 검색, 출력, 정정(訂正), 복구, 이용, 제공, 공개, 파기(破棄), 그 밖에 이와 유사한 행위를 말한다.  3. “정보주체”란 처리되는 정보에 의하여 알아볼 수 있는 사람으로서 그 정보의 주체가 되는 사람을 말한다. 4. “개인정보파일”이란 개인정보를 쉽게 검색할 수 있도록 일정한 규칙에 따라 체계적으로 배열하거나 구성한 개인 정보의 집합물(集合物)을 말한다.  5.

In [10]:
def clean_article_text(text):
    # 1. 한자 제거 (유니코드 범위: \u4e00-\u9fff)
    text = re.sub(r'[\u4e00-\u9fff]', '', text)

    # 2. 반복되는 출처 문구 제거
    text = re.sub(r'법제처\s+\d+\s+국가법령정보센터\s*개인정보 보호법', '', text)
    text = re.sub(r'법제처\s+\d+\s+국가법령정보센터', '', text)
    text = re.sub(r'국가법령정보센터\s*개인정보 보호법', '', text)
    text = re.sub(r'법제처', '', text)
    text = re.sub(r'국가법령정보센터', '', text)

    # 3. <개정 2023. 3. 14.> 또는 <신설 2023. 3. 14.> 등 각종 <>로 감싼 패턴 제거
    text = re.sub(r'<[^>]+>', '', text)

    # 4. [본조신설 2020. 2. 4.] 등 []로 감싼 패턴 제거
    text = re.sub(r'\[[^\]]+\]', '', text)

    # 5. ①, ②, ③ ... ⑳을 (1), (2), (3) ... (20)으로 변환
    circled_numbers = '①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'
    for idx, c in enumerate(circled_numbers, 1):
        text = text.replace(c, f'({idx})')

    # 앞뒤 불필요한 공백 줄이기
    text = re.sub(r'\s+', ' ', text).strip()

    # () 문자열 패턴 정리
    text = re.sub(r'\(\s*\)', '', text)
    
    return text

# 조문 단위로 분리 및 정제
def split_articles(text):
    # "제X조(제목)" 형식 찾기
    pattern = r'(제\d+조(?:의\d+)?\([^)]+\))'
    # 분리
    articles = re.split(pattern, text)
    results = []
    for i in range(1, len(articles), 2):
        article_num_title = articles[i]
        article_text = articles[i+1].strip().replace('\n', ' ')
        # '제X조(제목)'에서 번호와 제목 분리
        m = re.match(r'(제\d+조(?:의\d+)?)[(]([^)]+)[)]', article_num_title)
        if m:
            law_num = m.group(1)
            law_title = m.group(2)
            prompt = f"개인정보 보호법 {law_num} \"{law_title}\"에 대해서 설명하라."
            results.append({'instruction': prompt, 'output': clean_article_text(article_text)})
    return results

In [11]:
split_articles(full_text)

[{'instruction': '개인정보 보호법 제1조 "목적"에 대해서 설명하라.',
  'output': '이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인 의 존엄과 가치를 구현함을 목적으로 한다.'},
 {'instruction': '개인정보 보호법 제2조 "정의"에 대해서 설명하라.',
  'output': '이 법에서 사용하는 용어의 뜻은 다음과 같다. 1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다. 가. 성명, 주민등록번호 및 영상 등을 통하여 개인을 알아볼 수 있는 정보 나. 해당 정보만으로는 특정 개인을 알아볼 수 없더라도 다른 정보와 쉽게 결합하여 알아볼 수 있는 정보. 이 경우 쉽게 결합할 수 있는지 여부는 다른 정보의 입수 가능성 등 개인을 알아보는 데 소요되는 시간, 비용, 기술 등 을 합리적으로 고려하여야 한다. 다. 가목 또는 나목을 제1호의2에 따라 가명처리함으로써 원래의 상태로 복원하기 위한 추가 정보의 사용ㆍ결합 없이는 특정 개인을 알아볼 수 없는 정보(이하 “가명정보”라 한다) 1의2. “가명처리”란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의 방법으로 추가 정보가 없이는 특정 개인을 알아볼 수 없도록 처리하는 것을 말한다. 2. “처리”란 개인정보의 수집, 생성, 연계, 연동, 기록, 저장, 보유, 가공, 편집, 검색, 출력, 정정, 복구, 이용, 제공, 공개, 파기, 그 밖에 이와 유사한 행위를 말한다. 3. “정보주체”란 처리되는 정보에 의하여 알아볼 수 있는 사람으로서 그 정보의 주체가 되는 사람을 말한다. 4. “개인정보파일”이란 개인정보를 쉽게 검색할 수 있도록 일정한 규칙에 따라 체계적으로 배열하거나 구성한 개인 정보의 집합물을 말한다. 5. “개인정보처리자”란 업무를 목적으로 개인정보파일을 운용하기 위하여 스스로 또는 다른 사람을 통하여 개인정 보를 처리하는 공공기관, 법인, 단체 및 개인

In [12]:
def merge_consecutive_texts(texts):
    if not texts:
        return []
    
    merged = []
    current_ins = texts[0]['instruction']
    current_output = texts[0]['output']
    
    for text in texts[1:]:
        if text['instruction'] == current_ins:
            current_output += " " + text['output']
        else:
            merged.append({'instruction': current_ins, 'output': current_output})
            current_ins = text['instruction']
            current_output = text['output']
    
    merged.append({'instruction': current_ins, 'output': current_output})
    return merged

In [13]:
merge_consecutive_texts(split_articles(full_text))

[{'instruction': '개인정보 보호법 제1조 "목적"에 대해서 설명하라.',
  'output': '이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인 의 존엄과 가치를 구현함을 목적으로 한다.'},
 {'instruction': '개인정보 보호법 제2조 "정의"에 대해서 설명하라.',
  'output': '이 법에서 사용하는 용어의 뜻은 다음과 같다. 1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다. 가. 성명, 주민등록번호 및 영상 등을 통하여 개인을 알아볼 수 있는 정보 나. 해당 정보만으로는 특정 개인을 알아볼 수 없더라도 다른 정보와 쉽게 결합하여 알아볼 수 있는 정보. 이 경우 쉽게 결합할 수 있는지 여부는 다른 정보의 입수 가능성 등 개인을 알아보는 데 소요되는 시간, 비용, 기술 등 을 합리적으로 고려하여야 한다. 다. 가목 또는 나목을 제1호의2에 따라 가명처리함으로써 원래의 상태로 복원하기 위한 추가 정보의 사용ㆍ결합 없이는 특정 개인을 알아볼 수 없는 정보(이하 “가명정보”라 한다) 1의2. “가명처리”란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의 방법으로 추가 정보가 없이는 특정 개인을 알아볼 수 없도록 처리하는 것을 말한다. 2. “처리”란 개인정보의 수집, 생성, 연계, 연동, 기록, 저장, 보유, 가공, 편집, 검색, 출력, 정정, 복구, 이용, 제공, 공개, 파기, 그 밖에 이와 유사한 행위를 말한다. 3. “정보주체”란 처리되는 정보에 의하여 알아볼 수 있는 사람으로서 그 정보의 주체가 되는 사람을 말한다. 4. “개인정보파일”이란 개인정보를 쉽게 검색할 수 있도록 일정한 규칙에 따라 체계적으로 배열하거나 구성한 개인 정보의 집합물을 말한다. 5. “개인정보처리자”란 업무를 목적으로 개인정보파일을 운용하기 위하여 스스로 또는 다른 사람을 통하여 개인정 보를 처리하는 공공기관, 법인, 단체 및 개인

In [14]:
preprocessed_text = merge_consecutive_texts(split_articles(full_text))

In [16]:
preprocessed_text[0]

{'instruction': '개인정보 보호법 제1조 "목적"에 대해서 설명하라.',
 'output': '이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인 의 존엄과 가치를 구현함을 목적으로 한다.'}

In [17]:
# 개인정보 보호법 = Personal Information Protection Act (PIPA)
with open("../data/train_dataset_pipa.jsonl", "w", encoding="utf-8") as file:
    for text in preprocessed_text:
        json_obj = {"texts": text}
        json.dump(json_obj, file, ensure_ascii=False)
        file.write("\n") 